# Predicting House Sale Prices

In this project, I will build machine learning modeles to  predict the house sales prices and use techniques like Feature Engineering, Feature selection and K-Flod to improve the models I built.

I will work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. There are more information about why the data was collected [here](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627). Details regarding the different columns in the data can be found [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

## Summary of results:

I built a model which can be used to predict the house sale prices. Then I used several Feature Engineering techniques, Feature selection techniques and k parameter to set up a pipeline of functions that will let us quickly iterate on different models. __The RMES vaule has been decreased from 57088 at the beginning to 28976__.

## Reading in dataset

In [160]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

housing = pd.read_csv('AmesHousing.tsv', delimiter = '\t')

# primary exploration of the dataset
housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


## Creating primary functions

In this session, I am going to create some base functions

In [189]:
def transform_features(df):
    '''
    This function is designed to transform some features into useful features.
    For now it has no use
    '''
    return df

def select_features(df):
    '''
    This function is designed to select the feature column(s) and the traget column.
    '''
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    '''
    This function is designed to train and test the Linear Regression model
    '''
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include = ['integer', 'float'])
    numeric_test = test.select_dtypes(include = ['integer', 'float'])
    
    
    features = numeric_train.columns.tolist()
    features.remove('SalePrice')
    lr = LinearRegression()
    lr.fit(numeric_train[features], numeric_train['SalePrice'])
    predictions = lr.predict(numeric_test[features])
    rmse = np.sqrt(mean_squared_error(predictions, numeric_test['SalePrice']))
    return rmse

# primary test of the functions
transformed_housing = transform_features(housing)
selected_housing = select_features(transformed_housing)
rmse = train_and_test(selected_housing)

print(rmse)

57088.25161263909


## Feature Engineering


### Missing values
Handle missing values:
- All columns:
    - Drop any with 5% or more missing values __for now__.
- Text columns:
    - Drop any with 1 or more missing values __for now__.
- Numerical columns:
    - For columns with missing values, fill in with the most common value in that column
    

1: All columns: Drop any with 5% or more missing values for now.

In [162]:
# find out the number of missing values in each column
num_missing = housing.isnull().sum()

In [163]:
# filter out columns with 5% or more missing values
drop_missing_cols = num_missing[num_missing/len(housing) >= 0.05].index
housing = housing.drop(drop_missing_cols, axis=1)

2: Text columns: Drop any with 1 or more missing values for now.

In [164]:
# filter out text columns 
text_cols_mv = housing.select_dtypes(include=['object']).isnull().sum()

# filter out text columns with 1 or more missing values
drop_text_cols = text_cols_mv[text_cols_mv > 0].index
housing = housing.drop(drop_text_cols, axis=1)

3: Numerical columns: For columns with missing values, fill in with the most common value in that column

In [165]:
# filter out numerical columns which need to be fixed
numeric_cols_mv = housing.select_dtypes(include=['integer', 'float']).isnull().sum()
fixing_num_cols = numeric_cols_mv[numeric_cols_mv > 0]
fixing_num_cols

Mas Vnr Area      23
BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Cars        1
Garage Area        1
dtype: int64

In [166]:
# find out common value in those columns
replacing_value = housing[fixing_num_cols.index].mode().to_dict(orient='records')[0]

# replace missing vlaues
housing = housing.fillna(replacing_value)

# verify results
housing.isnull().sum().value_counts()

0    64
dtype: int64

### Creating new features by combining other features

In [167]:
# create new column year_before_sale
housing['year_before_sale'] = housing['Yr Sold'] - housing['Year Built']

# confirm there is no problemic data
housing[housing['year_before_sale'] < 0 ]

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,year_before_sale
2180,2181,908154195,20,RL,39290,Pave,IR1,Bnk,AllPub,Inside,...,0,0,0,17000,10,2007,New,Partial,183850,-1


In [168]:
# drop problemic row
housing = housing.drop([2180], axis = 0)

# confirm there is no problemic data
housing[housing['year_before_sale'] < 0 ]

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,year_before_sale


In [169]:
# create new column year_since_remod
housing['year_since_remod'] = housing['Yr Sold'] - housing['Year Remod/Add']

# confirm there is no problemic data
housing[housing['year_since_remod'] < 0 ]

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,year_before_sale,year_since_remod
1702,1703,528120010,60,RL,16659,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,6,2007,New,Partial,260116,0,-1
2181,2182,908154205,60,RL,40094,Pave,IR1,Bnk,AllPub,Inside,...,0,0,0,10,2007,New,Partial,184750,0,-1


In [170]:
# drop problemic row
housing = housing.drop([1702, 2181], axis = 0)

# confirm there is no problemic data
housing[housing['year_since_remod'] < 0 ]

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,year_before_sale,year_since_remod


In [171]:
# drop useless year columns
housing = housing.drop(['Yr Sold', 'Year Built', 'Year Remod/Add'], axis = 1)

### Dropping columns which have are not useful

In [172]:
## drop columns that aren't useful for ML
housing = housing.drop(['PID', 'Order'], axis=1)

## drop columns which about the final sale
housing = housing.drop(['Mo Sold', 'Sale Condition', 'Sale Type'], axis=1)

### Low variance numerical features

In [173]:
# check variance of numerical features
numeric_cols = housing.select_dtypes(include=['integer', 'float']).columns
unit_housing = (
                housing[numeric_cols] - housing[numeric_cols].min()
                ) / (
                housing[numeric_cols].max() - housing[numeric_cols].min())

print(unit_housing.var().sort_values())

Misc Val            0.000928
Lot Area            0.001345
Low Qual Fin SF     0.001896
Pool Area           0.001982
3Ssn Porch          0.002452
Enclosed Porch      0.004020
Total Bsmt SF       0.005022
Kitchen AbvGr       0.005097
BsmtFin SF 1        0.006348
1st Flr SF          0.007742
Wood Deck SF        0.007844
Open Porch SF       0.008018
Gr Liv Area         0.008794
Screen Porch        0.009491
Bedroom AbvGr       0.010712
SalePrice           0.011593
Mas Vnr Area        0.012250
BsmtFin SF 2        0.012297
Bsmt Half Bath      0.014967
TotRms AbvGrd       0.016976
Full Bath           0.019077
Overall Cond        0.019320
Garage Area         0.020827
Garage Cars         0.023113
Overall Qual        0.024461
Fireplaces          0.026216
Bsmt Full Bath      0.030597
Bsmt Unf SF         0.035351
2nd Flr SF          0.042953
year_before_sale    0.049585
MS SubClass         0.062954
Half Bath           0.063125
year_since_remod    0.120806
dtype: float64


There is no numerical feature with variance equal to 0.3 or lower.

## Updating transform_features()

In [174]:
def transform_features(df):
    '''
    This function is designed to clean data and transform some features into useful features.
    '''
    # find out the number of missing values in each column
    num_missing = df.isnull().sum()
    # filter out columns with 5% or more missing values
    drop_missing_cols = num_missing[num_missing/len(df) >= 0.05].index
    df = df.drop(drop_missing_cols, axis=1)
    
    # filter out text columns 
    text_cols_mv = df.select_dtypes(include=['object']).isnull().sum()
    # filter out text columns with 1 or more missing values
    drop_text_cols = text_cols_mv[text_cols_mv > 0].index
    df = df.drop(drop_text_cols, axis=1)
    
    # filter out numerical columns which need to be fixed
    numeric_cols_mv = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    fixing_num_cols = numeric_cols_mv[numeric_cols_mv > 0]
    # find out common value in those columns
    replacing_value = df[fixing_num_cols.index].mode().to_dict(orient='records')[0]
    # replace missing vlaues
    df = df.fillna(replacing_value)
    
    # create new column year_before_sale
    df['year_before_sale'] = df['Yr Sold'] - df['Year Built']
    
    # create new column year_since_remod
    df['year_since_remod'] = df['Yr Sold'] - df['Year Remod/Add']

    # drop problemic row
    df = df.drop([2180, 1702, 2181], axis = 0)
    
    # drop useless columns
    df = df.drop(['Yr Sold', 'Year Built', 'Year Remod/Add', 'PID', 'Order', 
                            'Mo Sold', 'Sale Condition', 'Sale Type'], axis = 1)
    return df

def select_features(df):
    '''
    This function is designed to select the feature column(s).
    '''
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    '''
    This function is designed to train and test the Linear Regression model
    '''
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include = ['integer', 'float'])
    numeric_test = test.select_dtypes(include = ['integer', 'float'])
    
    
    features = numeric_train.columns.tolist()
    features.remove('SalePrice')
    lr = LinearRegression()
    lr.fit(numeric_train[features], numeric_train['SalePrice'])
    predictions = lr.predict(numeric_test[features])
    rmse = np.sqrt(mean_squared_error(predictions, numeric_test['SalePrice']))
    return rmse

housing = pd.read_csv('AmesHousing.tsv', delimiter = '\t')
transformed_housing = transform_features(housing)
selected_housing = select_features(transformed_housing)
rmse = train_and_test(selected_housing)

print(rmse)

55275.36731241307


## Feature selection
In this session, I am going to select the suitable features for the model


### Correlation heatmap
First, I will use a correlation heatmap matrix to examine and find out the numerical features which correlate strongly with the target column, SalePrice.

In [175]:
numeric_housing = transformed_housing.select_dtypes(include=['integer', 'float'])
abs_corr_coeffs = numeric_housing.corr()['SalePrice'].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2        0.006127
Misc Val            0.019273
3Ssn Porch          0.032268
Bsmt Half Bath      0.035875
Low Qual Fin SF     0.037629
Pool Area           0.068438
MS SubClass         0.085128
Overall Cond        0.101540
Screen Porch        0.112280
Kitchen AbvGr       0.119760
Enclosed Porch      0.128685
Bedroom AbvGr       0.143916
Bsmt Unf SF         0.182751
Lot Area            0.267520
2nd Flr SF          0.269601
Bsmt Full Bath      0.276258
Half Bath           0.284871
Open Porch SF       0.316262
Wood Deck SF        0.328183
BsmtFin SF 1        0.439284
Fireplaces          0.474831
TotRms AbvGrd       0.498574
Mas Vnr Area        0.506983
year_since_remod    0.534985
Full Bath           0.546118
year_before_sale    0.558979
1st Flr SF          0.635185
Garage Area         0.641425
Total Bsmt SF       0.644012
Garage Cars         0.648361
Gr Liv Area         0.717596
Overall Qual        0.801206
SalePrice           1.000000
Name: SalePrice, dtype: float64

For now, only columns with correlation coefficient of larger than 0.4 with SalePrice will be kept. (arbitrary, worth experimenting later!)

In [176]:
# drop columns with correlation coefficient of lesser than 0.4 with SalePrice
transformed_housing = transformed_housing.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)

### Converting to the categorical data type
I will select some text columns and convert them to categorical data type.

In [177]:
# select text columns
text_cols = transformed_housing.select_dtypes(include=['object']).columns

text_cols

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Exter Qual', 'Exter Cond',
       'Foundation', 'Heating', 'Heating QC', 'Central Air', 'Kitchen Qual',
       'Functional', 'Paved Drive'],
      dtype='object')

From the documentation, I believe the following columns are meant to be categorical

In [178]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [179]:
# filter the text list with the categorical column list
transform_cat_cols = []
for col in text_cols:
    if col in nominal_features:
        transform_cat_cols.append(col)

# compute the unique value in each slected column
uniqueness_counts = transformed_housing[
    transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()

uniqueness_counts

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

Only columns with 10 or lesser unique values will be kept. (arbitrary, worth experimenting later!)

In [180]:
# drop text columns which have more than 10 unique values
drop_text_cols = uniqueness_counts[uniqueness_counts > 10].index
transformed_housing = transformed_housing.drop(drop_text_cols, axis=1)

# convert to the categorical data type
text_cols = transformed_housing.select_dtypes(include=['object']).columns
for col in text_cols:
     transformed_housing[col] = transformed_housing[col].astype('category')

### Low variance text features

In [181]:
# examinate feature variance
low_var_features = []
for col in text_cols:
    if transformed_housing[col].value_counts()[0]/len(transformed_housing) > 0.95:
        print(transformed_housing[col].value_counts())
        text_cols = text_cols.drop(col)
        low_var_features.append(col)

AllPub    2924
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64
Gtl    2786
Mod     125
Sev      16
Name: Land Slope, dtype: int64


Categorical columns which have a few unique values but more than 95% of the values in the column belong to a specific category are considered as low variance features. The columns will be removed.

In [182]:
# remove low variance features
transformed_housing = transformed_housing.drop(low_var_features, axis=1)

# apply dummy coding for selected columns
for col in text_cols:
    col_dummy = pd.get_dummies(transformed_housing[col])
    transformed_housing = pd.concat([transformed_housing, col_dummy], axis=1)
    del transformed_housing[col]

## Update select_features()

In [183]:
def transform_features(df):
    '''
    This function is designed to clean data and transform some features into useful features.
    '''
    # find out the number of missing values in each column
    num_missing = df.isnull().sum()
    # filter out columns with 5% or more missing values
    drop_missing_cols = num_missing[num_missing/len(df) >= 0.05].index
    df = df.drop(drop_missing_cols, axis=1)
    
    # filter out text columns 
    text_cols_mv = df.select_dtypes(include=['object']).isnull().sum()
    # filter out text columns with 1 or more missing values
    drop_text_cols = text_cols_mv[text_cols_mv > 0].index
    df = df.drop(drop_text_cols, axis=1)
    
    # filter out numerical columns which need to be fixed
    numeric_cols_mv = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    fixing_num_cols = numeric_cols_mv[numeric_cols_mv > 0]
    # find out common value in those columns
    replacing_value = df[fixing_num_cols.index].mode().to_dict(orient='records')[0]
    # replace missing vlaues
    df = df.fillna(replacing_value)
    
    # create new column year_before_sale
    df['year_before_sale'] = df['Yr Sold'] - df['Year Built']
    
    # create new column year_since_remod
    df['year_since_remod'] = df['Yr Sold'] - df['Year Remod/Add']

    # drop problemic row
    df = df.drop([2180, 1702, 2181], axis = 0)
    
    # drop useless columns
    df = df.drop(['Yr Sold', 'Year Built', 'Year Remod/Add', 'PID', 'Order', 
                            'Mo Sold', 'Sale Condition', 'Sale Type'], axis = 1)
    return df

def select_features(df):
    '''
    This function is designed to select the feature column(s).
    '''
    # drop columns with correlation coefficient of lesser than 0.4 with SalePrice
    numeric_housing = df.select_dtypes(include=['integer', 'float'])
    abs_corr_coeffs = numeric_housing.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)
    
    # select text columns
    text_cols = df.select_dtypes(include=['object']).columns
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    # filter the text list with the categorical column list
    transform_cat_cols = []
    for col in text_cols:
        if col in nominal_features:
            transform_cat_cols.append(col)
    # compute the unique value in each slected column
    uniqueness_counts = transformed_housing[
    transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    # drop text columns which have more than 10 unique values
    drop_text_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_text_cols, axis=1)
    # convert to the categorical data type
    text_cols = df.select_dtypes(include=['object']).columns
    for col in text_cols:
         df[col] = df[col].astype('category')
            
    # examinate feature variance
    low_var_features = []
    for col in text_cols:
        if df[col].value_counts()[0]/len(df) > 0.95:
            text_cols = text_cols.drop(col)
            low_var_features.append(col)
            
    # remove low variance features
    df = df.drop(low_var_features, axis=1)

    # apply dummy coding for selected columns
    for col in text_cols:
        col_dummy = pd.get_dummies(df[col])
        df = pd.concat([df, col_dummy], axis=1)
        del df[col]
    
    return df

def train_and_test(df):
    '''
    This function is designed to train and test the Linear Regression model
    '''
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include = ['integer', 'float'])
    numeric_test = test.select_dtypes(include = ['integer', 'float'])
    
    
    features = numeric_train.columns.tolist()
    features.remove('SalePrice')
    lr = LinearRegression()
    lr.fit(numeric_train[features], numeric_train['SalePrice'])
    predictions = lr.predict(numeric_test[features])
    rmse = np.sqrt(mean_squared_error(predictions, numeric_test['SalePrice']))
    return rmse

housing = pd.read_csv('AmesHousing.tsv', delimiter = '\t')
transformed_housing = transform_features(housing)
selected_housing = select_features(transformed_housing)
rmse = train_and_test(selected_housing)

print(rmse)

36623.53562910476


## Update train_and_test() by including k parameter

In [192]:
def transform_features(df):
    '''
    This function is designed to clean data and transform some features into useful features.
    '''
    # find out the number of missing values in each column
    num_missing = df.isnull().sum()
    # filter out columns with 5% or more missing values
    drop_missing_cols = num_missing[num_missing/len(df) >= 0.05].index
    df = df.drop(drop_missing_cols, axis=1)
    
    # filter out text columns 
    text_cols_mv = df.select_dtypes(include=['object']).isnull().sum()
    # filter out text columns with 1 or more missing values
    drop_text_cols = text_cols_mv[text_cols_mv > 0].index
    df = df.drop(drop_text_cols, axis=1)
    
    # filter out numerical columns which need to be fixed
    numeric_cols_mv = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    fixing_num_cols = numeric_cols_mv[numeric_cols_mv > 0]
    # find out common value in those columns
    replacing_value = df[fixing_num_cols.index].mode().to_dict(orient='records')[0]
    # replace missing vlaues
    df = df.fillna(replacing_value)
    
    # create new column year_before_sale
    df['year_before_sale'] = df['Yr Sold'] - df['Year Built']
    
    # create new column year_since_remod
    df['year_since_remod'] = df['Yr Sold'] - df['Year Remod/Add']

    # drop problemic row
    df = df.drop([2180, 1702, 2181], axis = 0)
    
    # drop useless columns
    df = df.drop(['Yr Sold', 'Year Built', 'Year Remod/Add', 'PID', 'Order', 
                            'Mo Sold', 'Sale Condition', 'Sale Type'], axis = 1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10, variance_threshold=0.95):
    '''
    This function is designed to select the feature column(s).
    '''
    # drop columns with correlation coefficient of lesser than coeff_threshold with SalePrice
    numeric_housing = df.select_dtypes(include=['integer', 'float'])
    abs_corr_coeffs = numeric_housing.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    # select text columns
    text_cols = df.select_dtypes(include=['object']).columns
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    # filter the text list with the categorical column list
    transform_cat_cols = []
    for col in text_cols:
        if col in nominal_features:
            transform_cat_cols.append(col)
    # compute the unique value in each slected column
    uniqueness_counts = transformed_housing[
        transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    # drop text columns which have more than uniq_threshold
    drop_text_cols = uniqueness_counts[uniqueness_counts > uniq_threshold].index
    df = df.drop(drop_text_cols, axis=1)
    # convert to the categorical data type
    text_cols = df.select_dtypes(include=['object']).columns
    for col in text_cols:
         df[col] = df[col].astype('category')
            
    # examinate feature variance
    low_var_features = []
    for col in text_cols:
        if df[col].value_counts()[0]/len(df) > variance_threshold:
            text_cols = text_cols.drop(col)
            low_var_features.append(col)
            
    # remove low variance features
    df = df.drop(low_var_features, axis=1)

    # apply dummy coding for selected columns
    for col in text_cols:
        col_dummy = pd.get_dummies(df[col])
        df = pd.concat([df, col_dummy], axis=1)
        del df[col]
    
    return df

def train_and_test(df, k=0):
    '''
    This function is designed to train and test the Linear Regression model
    '''
    lr = LinearRegression()
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.tolist()
    features.remove('SalePrice')
    
    if k == 0:
    
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train['SalePrice'])
        predictions = lr.predict(test[features])
        rmse = np.sqrt(mean_squared_error(predictions, test['SalePrice']))
        
        return rmse
    
    if k == 1:
        
        # randomize the order of a DataFrame
        shuffled_df = df.sample(frac=1,)
        fold_one = shuffled_df[:1460]
        fold_two = shuffled_df[1460:]
        
        lr.fit(fold_one[features], fold_one['SalePrice'])
        predictions = lr.predict(fold_two[features])
        rmse_1 = np.sqrt(mean_squared_error(predictions, fold_two['SalePrice']))
        
        lr.fit(fold_two[features], fold_two['SalePrice'])
        predictions = lr.predict(fold_one[features])
        rmse_2 = np.sqrt(mean_squared_error(predictions, fold_one['SalePrice']))
        
        return np.mean([rmse_1, rmse_2])
    
    else:
        
        kf = KFold(n_splits=k, shuffle=True)
        rmses = []
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train['SalePrice'])
            predictions = lr.predict(test[features])
            rmse = np.sqrt(mean_squared_error(predictions, test['SalePrice']))
            rmses.append(rmse)
        avg_rmse = np.mean(rmses) 
        print(rmses)   
        return avg_rmse

housing = pd.read_csv('AmesHousing.tsv', delimiter = '\t')
transformed_housing = transform_features(housing)
selected_housing = select_features(transformed_housing, coeff_threshold=0.4, uniq_threshold=10, variance_threshold=0.95)
rmse = train_and_test(selected_housing, k=4)

print(rmse)

[25516.92158091976, 28525.929455058376, 25321.09118729429, 36540.67879765745]
28976.155255232472


## Conclusion:

In this project, I built a model which can be used to predict the house sale prices. Then I used several Feature Engineering techniques, Feature selection techniques and k parameter to set up a pipeline of functions that will let us quickly iterate on different models. The RMES vaule has been decreased from 57088 at the beginning to 28976.